In [5]:
!nodetool status

Datacenter: datacenter1
Status=Up/Down
|/ State=Normal/Leaving/Joining/Moving
--  Address     Load        Tokens  Owns (effective)  Host ID                               Rack 
UN  172.25.0.2  104.33 KiB  16      64.2%             bdbda029-3c21-493e-827c-15cb495b7b52  rack1
UN  172.25.0.4  104.34 KiB  16      69.7%             cf548ac5-235e-4e43-8336-15d1563d81e9  rack1
UN  172.25.0.3  104.32 KiB  16      66.2%             e5d4c00b-49eb-4fb0-b2df-f0042e103afa  rack1



In [6]:
from cassandra.cluster import Cluster
cluster = Cluster(['p6-db-1', 'p6-db-2', 'p6-db-3'])
cass = cluster.connect()

In [7]:
# cass.execute("drop table if exists stations")

In [7]:
cass.execute("DROP KEYSPACE IF EXISTS weather")
cass.execute("""
    CREATE KEYSPACE weather
    WITH replication = {'class': 'SimpleStrategy', 'replication_factor': 3}
""")

cass.set_keyspace("weather")
cass.execute("""
    CREATE TYPE station_record (
        tmin INT,
        tmax INT
    )
""")

cass.execute("""
    CREATE TABLE stations (
        id TEXT,
        name TEXT STATIC,
        date DATE,
        record weather.station_record,
        PRIMARY KEY ((id), date)
    ) WITH CLUSTERING ORDER BY (date ASC)
""")

In [8]:
#q1
print(cass.execute("describe table weather.stations").one().create_statement)

CREATE TABLE weather.stations (
    id text,
    date date,
    name text static,
    record station_record,
    PRIMARY KEY (id, date)
) WITH CLUSTERING ORDER BY (date ASC)
    AND additional_write_policy = '99p'
    AND bloom_filter_fp_chance = 0.01
    AND caching = {'keys': 'ALL', 'rows_per_partition': 'NONE'}
    AND cdc = false
    AND comment = ''
    AND compaction = {'class': 'org.apache.cassandra.db.compaction.SizeTieredCompactionStrategy', 'max_threshold': '32', 'min_threshold': '4'}
    AND compression = {'chunk_length_in_kb': '16', 'class': 'org.apache.cassandra.io.compress.LZ4Compressor'}
    AND memtable = 'default'
    AND crc_check_chance = 1.0
    AND default_time_to_live = 0
    AND extensions = {}
    AND gc_grace_seconds = 864000
    AND max_index_interval = 2048
    AND memtable_flush_period_in_ms = 0
    AND min_index_interval = 128
    AND read_repair = 'BLOCKING'
    AND speculative_retry = '99p';


In [9]:
from pyspark.sql import SparkSession
spark = (SparkSession.builder
         .appName("p6")
         .config('spark.jars.packages', 'com.datastax.spark:spark-cassandra-connector_2.12:3.4.0')
         .config("spark.sql.extensions", "com.datastax.spark.connector.CassandraSparkExtensions")
         .getOrCreate())

:: loading settings :: url = jar:file:/usr/local/lib/python3.10/dist-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /root/.ivy2/cache
The jars for the packages stored in: /root/.ivy2/jars
com.datastax.spark#spark-cassandra-connector_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-8ca961df-fed6-4144-b94c-ce1917a9b5b8;1.0
	confs: [default]
	found com.datastax.spark#spark-cassandra-connector_2.12;3.4.0 in central
	found com.datastax.spark#spark-cassandra-connector-driver_2.12;3.4.0 in central
	found com.datastax.oss#java-driver-core-shaded;4.13.0 in central
	found com.datastax.oss#native-protocol;1.5.0 in central
	found com.datastax.oss#java-driver-shaded-guava;25.1-jre-graal-sub-1 in central
	found com.typesafe#config;1.4.1 in central
	found org.slf4j#slf4j-api;1.7.26 in central
	found io.dropwizard.metrics#metrics-core;4.1.18 in central
	found org.hdrhistogram#HdrHistogram;2.1.12 in central
	found org.reactivestreams#reactive-streams;1.0.3 in central
	found com.github.stephenc.jcip#jcip-annotations;1.0-1 in central
	found com.gith

In [10]:
from pyspark.sql.functions import col, expr
df = spark.read.text("ghcnd-stations.txt")
pandas_df1 = df.withColumn("ID", expr("substring(value, 0, 11)"))
pandas_df2 = pandas_df1.withColumn("STATE", expr("substring(value, 39, 2)"))
pandas_df3 = pandas_df2.withColumn("NAME", expr("substring(value, 42, 28)"))

pandas_wisco = pandas_df3.filter(pandas_df3['STATE'] == 'WI').collect()

prepared = cass.prepare("INSERT INTO weather.stations(id, name) VALUES(?,?)")

for row in pandas_wisco:
    station_id = row["ID"]
    station_name = row["NAME"]
    cass.execute(prepared,(station_id,station_name))

In [11]:
#q2
name = cass.execute("""
    select name
    from weather.stations
    where id = 'USW00014837' 
""")

for row in name:
    station_name = row.name
station_name.strip()

'MADISON DANE CO RGNL AP'

In [12]:
#q3
token = cass.execute("""
    select token(id) as token_id
    from weather.stations
    where id = 'USC00470273'
    """)

for row in token:
    token_id = row.token_id
    
token_id

-9014250178872933741

In [13]:
#q4
import subprocess

# run nodetool ring command
nodetool_output = subprocess.check_output(["nodetool", "ring"]).decode("utf-8").split('\n')

# token value for USC00470273
target_token = "-9014250178872933741"

token_lines = [line.strip() for line in nodetool_output[5:-6] if line.strip()]
# print(token_lines)

ring_tokens = [int(line.split()[-1]) for line in token_lines]
#print(ring_tokens)

next_token = None

# check for wrap around
if int(target_token) >= max(ring_tokens):
    next_token = min(ring_tokens)
else:
    for token in sorted(ring_tokens):
        if token > int(target_token):
            next_token = token
            break

next_token

-8884495976480270205

In [14]:
import zipfile

with zipfile.ZipFile('records.zip', 'r') as zip_ref:
  zip_ref.extractall('')

In [15]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, from_unixtime

spark = SparkSession.builder.appName("stations").getOrCreate()

parquet_path = 'records.parquet'
df = spark.read.parquet(parquet_path)

# df.show()

pivoted_df = (
    df.groupBy("station", "date")
    .pivot("element")
    .agg({"value": "first"})
    .drop("null")
    .select("station", "date", col("TMIN").alias("tmin"), col("TMAX").alias("tmax"))
)

# pivoted_df.show()

import grpc
import station_pb2
import station_pb2_grpc

formatted_df = pivoted_df.withColumn("date", from_unixtime("date", "yyyy-MM-dd"))

with grpc.insecure_channel('localhost:5440') as channel:
    stub = station_pb2_grpc.StationStub(channel)
    
    for row in formatted_df.rdd.collect():
        request = station_pb2.RecordTempsRequest(
            station=row.station,
            date=row.date,
            tmin=int(row.tmin),
            tmax=int(row.tmax)
        )
        
        # Make gRPC call to insert the measurements
        response = stub.RecordTemps(request)
        if response.error:
            print(f"Error inserting data for station {row.station} on date {row.date}: {response.error}")

formatted_df.show()

23/11/27 02:59:38 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.
23/11/27 02:59:42 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


+-----------+----------+------+------+
|    station|      date|  tmin|  tmax|
+-----------+----------+------+------+
|USW00014898|1970-08-23|-166.0| -71.0|
|USW00014839|1970-08-23| 117.0| 194.0|
|USW00014839|1970-08-23|  83.0| 150.0|
|USW00014839|1970-08-23|  11.0|  83.0|
|USW00014839|1970-08-23| 139.0| 261.0|
|USR0000WDDG|1970-08-23|-106.0| -39.0|
|USR0000WDDG|1970-08-23|-178.0| -56.0|
|USW00014837|1970-08-23| -88.0| -38.0|
|USR0000WDDG|1970-08-23|-150.0|-106.0|
|USW00014839|1970-08-23|   0.0|  39.0|
|USW00014839|1970-08-23| 189.0| 222.0|
|USW00014839|1970-08-23| 200.0| 294.0|
|USW00014837|1970-08-23| 200.0| 322.0|
|USW00014898|1970-08-23|-116.0| -60.0|
|USW00014839|1970-08-23| 156.0| 233.0|
|USR0000WDDG|1970-08-23|-128.0| -61.0|
|USR0000WDDG|1970-08-23|-117.0| -33.0|
|USR0000WDDG|1970-08-23| -17.0|  50.0|
|USW00014898|1970-08-23| 156.0| 256.0|
|USW00014837|1970-08-23| 117.0| 256.0|
+-----------+----------+------+------+
only showing top 20 rows



In [18]:
#q5
with grpc.insecure_channel('localhost:5440') as channel:
    stub = station_pb2_grpc.StationStub(channel)

    # find the max temperature for station 'USW00014837'
    max_temperature_row = formatted_df.filter(col("station") == "USW00014837").select("tmax").agg({"tmax": "max"}).collect()
    max_temperature = max_temperature_row[0]["max(tmax)"]
    print(max_temperature)
    # make gRPC call to get the max temperature for the station
    request = station_pb2.StationMaxRequest(station="USW00014837")
    response = stub.StationMax(request)

    print(response)
    
    # compare the results
    if response.tmax == max_temperature:
        print(f"The max temperature for station USW00014837 is {max_temperature}")
    else:
        print("Error: The calculated max temperature does not match the gRPC response.")

356.0
tmax: 17

Error: The calculated max temperature does not match the gRPC response.


In [19]:
spark.read.format("org.apache.spark.sql.cassandra").option("spark.cassandra.connection.host", "p6-db-1,p6-db-2,p6-db-3").option("keyspace", "weather").option("table", "stations").load().createOrReplaceTempView("stations")

In [20]:
#q6
spark.catalog.listTables()

[Table(name='stations', catalog=None, namespace=[], description=None, tableType='TEMPORARY', isTemporary=True)]

In [16]:
#spark.sql("select * from stations").show()

In [21]:
#q7
temperature_diff_df = formatted_df.select(
    "station",
    ((col("tmax") - col("tmin")).alias("temp_diff"))
)
avg_diff_per_station = temperature_diff_df.groupBy("station").agg({"temp_diff": "avg"})
result_dict = avg_diff_per_station.rdd.collectAsMap()

result_dict

{'USW00014839': 89.6986301369863,
 'USW00014837': 105.62739726027397,
 'USR0000WDDG': 102.06849315068493,
 'USW00014898': 102.93698630136986}

23/11/27 03:06:01 WARN ChannelPool: [s0|p6-db-2/172.25.0.4:9042]  Error while opening new channel (ConnectionInitException: [s0|connecting...] Protocol initialization request, step 1 (STARTUP {CQL_VERSION=3.0.0, DRIVER_NAME=DataStax Java driver for Apache Cassandra(R), DRIVER_VERSION=4.13.0, CLIENT_ID=b19bc2a3-e751-42e9-b763-9d68912cf089, APPLICATION_NAME=Spark-Cassandra-Connector-local-1701053904436}): failed to send request (java.nio.channels.NotYetConnectedException))
23/11/27 03:06:08 WARN ChannelPool: [s0|p6-db-2/172.25.0.4:9042]  Error while opening new channel (ConnectionInitException: [s0|connecting...] Protocol initialization request, step 1 (STARTUP {CQL_VERSION=3.0.0, DRIVER_NAME=DataStax Java driver for Apache Cassandra(R), DRIVER_VERSION=4.13.0, CLIENT_ID=b19bc2a3-e751-42e9-b763-9d68912cf089, APPLICATION_NAME=Spark-Cassandra-Connector-local-1701053904436}): failed to send request (java.nio.channels.NotYetConnectedException))


In [22]:
#q8
!nodetool status

23/11/27 03:06:18 WARN ChannelPool: [s0|p6-db-2/172.25.0.4:9042]  Error while opening new channel (ConnectionInitException: [s0|connecting...] Protocol initialization request, step 1 (STARTUP {CQL_VERSION=3.0.0, DRIVER_NAME=DataStax Java driver for Apache Cassandra(R), DRIVER_VERSION=4.13.0, CLIENT_ID=b19bc2a3-e751-42e9-b763-9d68912cf089, APPLICATION_NAME=Spark-Cassandra-Connector-local-1701053904436}): failed to send request (java.nio.channels.NotYetConnectedException))


Datacenter: datacenter1
Status=Up/Down
|/ State=Normal/Leaving/Joining/Moving
--  Address     Load       Tokens  Owns (effective)  Host ID                               Rack 
UN  172.25.0.2  86.03 KiB  16      100.0%            bdbda029-3c21-493e-827c-15cb495b7b52  rack1
DN  172.25.0.4  86.04 KiB  16      100.0%            cf548ac5-235e-4e43-8336-15d1563d81e9  rack1
UN  172.25.0.3  86.01 KiB  16      100.0%            e5d4c00b-49eb-4fb0-b2df-f0042e103afa  rack1



In [29]:
#q9
channel = grpc.insecure_channel('localhost:5440')
stub = station_pb2_grpc.StationStub(channel)
request = station_pb2.StationMaxRequest(station="USW00014837")
reply = stub.StationMax(request)
print(reply.error)

need 3 replicas, but only have 2


23/11/27 03:09:17 WARN ChannelPool: [s0|p6-db-2/172.25.0.4:9042]  Error while opening new channel (ConnectionInitException: [s0|connecting...] Protocol initialization request, step 1 (STARTUP {CQL_VERSION=3.0.0, DRIVER_NAME=DataStax Java driver for Apache Cassandra(R), DRIVER_VERSION=4.13.0, CLIENT_ID=b19bc2a3-e751-42e9-b763-9d68912cf089, APPLICATION_NAME=Spark-Cassandra-Connector-local-1701053904436}): failed to send request (com.datastax.oss.driver.shaded.netty.channel.StacklessClosedChannelException))


In [36]:
#q10
channel = grpc.insecure_channel('localhost:5440')
stub = station_pb2_grpc.StationStub(channel)
request = station_pb2.RecordTempsRequest(
    station="US11262023",
    date="2023-11-26",
    tmin=1,
    tmax=100
)
reply = stub.RecordTemps(request)
reply.error

''

23/11/27 03:15:09 WARN ChannelPool: [s0|p6-db-2/172.25.0.4:9042]  Error while opening new channel (ConnectionInitException: [s0|connecting...] Protocol initialization request, step 1 (STARTUP {CQL_VERSION=3.0.0, DRIVER_NAME=DataStax Java driver for Apache Cassandra(R), DRIVER_VERSION=4.13.0, CLIENT_ID=b19bc2a3-e751-42e9-b763-9d68912cf089, APPLICATION_NAME=Spark-Cassandra-Connector-local-1701053904436}): failed to send request (com.datastax.oss.driver.shaded.netty.channel.StacklessClosedChannelException))
